In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
import time
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver


In [2]:
load_dotenv()

llm = ChatOpenAI()

In [3]:
class CrashState(TypedDict) :
    input : str
    step1 : str
    step2 : str
    step3 : str


In [4]:
# Define Steps 
def step_1(state: CrashState) :
    print("Step 1 executed")
    return {'step1': "done"}
def step_2(state: CrashState) :
    print("Step 2 hanging...now manually interpt the code")
    time.sleep(30)
    return {'step2': "done"}
def step_3(state: CrashState) :
    print("Step 3 executed")
    return {'step3': "done"}


In [5]:
# Build the graph 
builder = StateGraph(CrashState) 
builder.add_node("step_1",step_1)
builder.add_node("step_2",step_2)
builder.add_node("step_3",step_3)

builder.add_edge(START, 'step_1')
builder.add_edge('step_1', 'step_2')
builder.add_edge('step_2', 'step_3')
builder.add_edge('step_3', END)

checkpointer = InMemorySaver()

workflow = builder.compile(checkpointer=checkpointer)

In [6]:
config = {'configurable' : {'thread_id' : 1}}
workflow.invoke({'input' : 'input'}, config = config)

Step 1 executed
Step 2 hanging...now manually interpt the code


KeyboardInterrupt: 

In [7]:
list(workflow.get_state_history(config))

[StateSnapshot(values={'input': 'input', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c127d-49c0-6dc7-8001-fd16a79c2e9a'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-11-14T07:02:06.336762+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c127d-49be-668b-8000-3843b4b4e2cf'}}, tasks=(PregelTask(id='319ae265-5529-2c6c-eb27-2a233808e274', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'input'}, next=('step_1',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c127d-49be-668b-8000-3843b4b4e2cf'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-11-14T07:02:06.335757+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c127d-49bb

In [9]:
workflow.get_state({"configurable" : {'thread_id' : 1, 'checkpoint_id' : '1f0c127d-49be-668b-8000-3843b4b4e2cf'}})

StateSnapshot(values={'input': 'input'}, next=('step_1',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0c127d-49be-668b-8000-3843b4b4e2cf'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-11-14T07:02:06.335757+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c127d-49bb-6fb8-bfff-2f565293026a'}}, tasks=(PregelTask(id='e4150e0b-2140-c6e5-9381-4523b515dfee', name='step_1', path=('__pregel_pull', 'step_1'), error=None, interrupts=(), state=None, result={'step1': 'done'}),), interrupts=())

In [11]:
# this will replay the step using the checkpoint, helpfull in debugging
workflow.invoke(None, config = {"configurable" : {'thread_id' : 1, 'checkpoint_id' : '1f0c127d-49be-668b-8000-3843b4b4e2cf'}})

Step 1 executed
Step 2 hanging...now manually interpt the code
Step 3 executed


{'input': 'input', 'step1': 'done', 'step2': 'done', 'step3': 'done'}

In [13]:
# this will also changes the history 
list(workflow.get_state_history(config=config))

[StateSnapshot(values={'input': 'input', 'step1': 'done', 'step2': 'done', 'step3': 'done'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c1289-46b3-6218-8003-450a6a3a70a6'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-11-14T07:07:28.139112+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c1289-46b0-6ac2-8002-e3a1b05cafdb'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'input', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c1289-46b0-6ac2-8002-e3a1b05cafdb'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-14T07:07:28.138105+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c1288-2893-6f26-8001-6d52eb7ca252'}}, tasks=(PregelTask(id='3fbf927f-ba5f-6d8c-1315-b73466ef0e68', name='s

# Update State

In [18]:
# To update the state in some step using its config_id
workflow.update_state({"configurable" : {'thread_id' : 1, 'checkpoint_id' : '1f0c127d-49be-668b-8000-3843b4b4e2cf', 'checkpoint_ns' : ""}}, {'input' : 'input-2'} )

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0c1297-a5f9-6158-8001-78df2f27e32e'}}

In [19]:
list(workflow.get_state_history(config))

[StateSnapshot(values={'input': 'input-2', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c1297-a5f9-6158-8001-78df2f27e32e'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-11-14T07:13:53.938876+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c127d-49be-668b-8000-3843b4b4e2cf'}}, tasks=(PregelTask(id='962afa0c-8021-78f6-607f-1c9ae6bbd684', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'input', 'step1': 'done', 'step2': 'done', 'step3': 'done'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c1289-46b3-6218-8003-450a6a3a70a6'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-11-14T07:07:28.139112+00:00', parent_config={'configurable': {'thread_id': '1', 'check

In [20]:
workflow.invoke(None, config = {'configurable':{"thread_id" : 1, 'checkpoint_id': '1f0c1297-a5f9-6158-8001-78df2f27e32e'}})

Step 2 hanging...now manually interpt the code
Step 3 executed


{'input': 'input-2', 'step1': 'done', 'step2': 'done', 'step3': 'done'}

In [ ]:
list(workflow.get_state_history(config))

[StateSnapshot(values={'input': 'input-2', 'step1': 'done', 'step2': 'done', 'step3': 'done'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c129e-a9f1-6b34-8003-9ad2d9804a92'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-11-14T07:17:02.260101+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c129e-a9ef-63e2-8002-0c71f2bb80df'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'input-2', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c129e-a9ef-63e2-8002-0c71f2bb80df'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-14T07:17:02.259094+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c1297-a5f9-6158-8001-78df2f27e32e'}}, tasks=(PregelTask(id='76b4010e-980d-91e5-222b-b2030c713d5f', nam

: 